In [37]:
import pandas as pd
import holidays
import datetime
from datetime import timedelta
import dateutil.parser as dparser

In [38]:
CLOCL_END = 18
CLOCL_INIT = 8
CLOCK_WORKING_DAY = CLOCL_END - CLOCL_INIT
SABADO = 5
DOMINGO = 6

In [39]:
HOLIDAYS = holidays.Brazil()['2021-01-01': '2031-12-31']

In [40]:
def is_dia_util(date:object):
    ans = date
    if date.weekday() == SABADO or date.weekday() == DOMINGO or date in HOLIDAYS:
        ans = is_dia_util(date + datetime.timedelta(days=1))
    return ans

In [41]:
def proximo_dia_util(date):
    def is_dia_util(date:object):
        ans = date
        if date.weekday() == SABADO or date.weekday() == DOMINGO or date in HOLIDAYS:
            ans = is_dia_util(date + datetime.timedelta(days=1))
        return ans
    
    return is_dia_util(date + datetime.timedelta(days=1))

In [42]:
def dias_uteis(date, days):
    for c in range(days):
        date = proximo_dia_util(date)
    return date

In [43]:
def calculo_prazo(sla, ano, mes, dia, hora, minuto=0, segundo=0):
    date = datetime.datetime(ano, mes, dia)  
    days_next=0

    if date.weekday() == SABADO or date.weekday() == DOMINGO or date in HOLIDAYS:
        day_working_clock = 0
    else:
        day_working_clock = (3600*CLOCL_END) - ((hora*3600)+(minuto*60)+ segundo)

    day_now = True if day_working_clock>sla*3600 else False

    if day_now:
        return date + datetime.timedelta(hours=sla)
    else:
        days_next+=1

    horas, minutos, segundos = segundos_to_clock((sla*3600) - day_working_clock)

    days_next+=horas//10
    horas = horas%10
    data_end = dias_uteis(date, days_next)

    delta = data_end + timedelta(hours=(horas+CLOCL_INIT), minutes=minutos, seconds=segundos)
    return delta

In [45]:
def data_limite(p):
    date = dparser.parse(str(p["DATA_ABERTURA"]))
    return calculo_prazo(p["SLA"], ano=date.year, mes=date.month, dia=date.day, hora=date.hour, minuto=date.minute, segundo=date.second)

In [46]:
def calculate_hour(date, sla):
    ano=date.year
    mes=date.month
    dia=date.day
    hora=date.hour
    minuto=date.minute
    segundo=date.second

    date = datetime.datetime(ano, mes, dia) 
     
    if date.weekday() == SABADO or date.weekday() == DOMINGO or date in HOLIDAYS:
        day_working_clock = 0
    else:
        day_working_clock = ((timedelta(hours = CLOCL_END) - timedelta(hours = hora, minutes=minuto, seconds=segundo)).seconds)
        
    dias = int((sla-day_working_clock)//CLOCK_WORKING_DAY)
    horas = ((sla-day_working_clock)%CLOCK_WORKING_DAY)//1
    minutos = (((sla-day_working_clock)%CLOCK_WORKING_DAY)%1)*60

    return int((dias*10)+horas), int(minutos)

In [47]:
def segundos_to_clock(segundos):
    return segundos//3600, (segundos%3600)//60,  (segundos%3600)%60

In [48]:
def calculate_hour_date_date(date, day_now):
    ano=date.year
    mes=date.month
    dia=date.day
    hora=date.hour
    minuto=date.minute
    segundo=date.second

    if is_dia_util(day_now) == day_now:
        day_working_clock = (3600*CLOCL_END) - ((day_now.hour*3600)+(day_now.minute*60)+ day_now.second)
        day_working_clock = day_working_clock if day_working_clock>0 else 0

    else:
        day_working_clock = 0

    count=1
    days_working = 0

    while True:
        new_date = day_now.date() + datetime.timedelta(days=count)
        new_date_is_util = is_dia_util(new_date)
        count+=1

        if new_date_is_util == new_date:
            days_working+=1

        else:
            days_working = days_working

        if new_date_is_util >= datetime.date(ano, mes, dia) and new_date_is_util == new_date:
            days_working-=1
            break

        elif new_date_is_util >= datetime.date(ano, mes, dia):
            break

    if date.weekday() != SABADO and date.weekday() != DOMINGO and date not in HOLIDAYS:
        horas_last_day = ((timedelta(hours = hora, minutes=minuto, seconds=segundo) - timedelta(hours = CLOCL_INIT)).seconds)
        horas_last_day = horas_last_day if timedelta(hours = hora, minutes=minuto, seconds=segundo) < timedelta(hours = CLOCL_END) else 10*3600

    else:
        horas_last_day = 0

    hora, minuto, segundo = segundos_to_clock(day_working_clock+horas_last_day)
    return ((days_working*10)+hora, minuto, segundo )

In [49]:
def prazo_em_horas(p):
    date = (dparser.parse(str(p['DATA_HORA_LIMITE'])) - datetime.datetime.now())

    if datetime.datetime.now() < dparser.parse(str(p['DATA_ABERTURA'])):
        hora, minuto, segundo  = calculate_hour_date_date(dparser.parse(str(p['DATA_ABERTURA'])), datetime.datetime.now())
        return f'{hora+p["SLA"]}:{minuto:02d}:{segundo:02d}'

    elif dparser.parse(str(p['DATA_HORA_LIMITE'])) < datetime.datetime.now():
        return 'VENCIDO'

    else:
        hora, minuto, segundo  = calculate_hour_date_date(datetime.datetime.now(), dparser.parse(str(p['DATA_ABERTURA'])))
        hora, minuto, segundo = segundos_to_clock((p["SLA"]*3600)-(hora*3600)-(minuto*3600)-segundo)
        return f'{hora:02d}:{minuto:02d}:{segundo:02d}'


In [50]:
def get_faixa(df):
    hora = df['PRAZO_EM_HORAS']
    if hora=='VENCIDO': return 'VENCIDO'
    if hora=='A INICIAR': return 'A INICIAR'
    
    hora = hora.split(":")
    h=int(hora[0])
    m=int(hora[1])
    if h == 0:
        if m < 30:
            return "<30 min"
        else:
            return ">=30min e <1h"
    if h > 7:
        return ">=8h"
    else:
        ans = ">=" +str(h)+"h "
        if m < 30:
            ans = ans + "e <"+str(h) + "h 30"
        else:
            ans = ans + "30 e <"+str(h+1) + "h"
    return ans

In [51]:
def format_date(date):#2021-08-31 12:30:12	to 26/08/2021  08:30:12
    date, clock = date.split(' ')
    date= date.split('-')
    return f'{date[2]}/{date[1]}/{date[0]} {clock}'

In [52]:
df= pd.read_excel("HORAS_AUDIT.xlsm", sheet_name = "BASE", usecols =["DATA_ABERTURA", "SLA"])
df.head()

,DATA_ABERTURA,SLA
0,2021-08-26 08:30:12,34
1,2021-09-17 12:53:19,500
2,2021-08-17 12:53:19,19
3,2021-08-17 12:53:19,12
4,2021-08-17 12:53:19,13


In [53]:
df['DATA_HORA_LIMITE'] = df.apply(data_limite, axis=1)


In [54]:
df['PRAZO_EM_HORAS'] = df.apply(prazo_em_horas, axis=1)


In [55]:
df['FAIXA'] = df.apply(get_faixa, axis=1)


In [56]:
df['DATA_HORA_LIMITE'] = df['DATA_HORA_LIMITE'].apply(lambda x: format_date(str(x)))
df['DATA_ABERTURA'] = df['DATA_ABERTURA'].apply(lambda x: format_date(str(x)))


In [57]:
df

,DATA_ABERTURA,SLA,DATA_HORA_LIMITE,PRAZO_EM_HORAS,FAIXA
0,26/08/2021 08:30:12,34,31/08/2021 12:30:12,64:30:12,>=8h
1,17/09/2021 12:53:19,500,26/11/2021 12:53:19,684:53:19,>=8h
2,17/08/2021 12:53:19,19,19/08/2021 11:53:19,VENCIDO,VENCIDO
3,17/08/2021 12:53:19,12,18/08/2021 14:53:19,VENCIDO,VENCIDO
4,17/08/2021 12:53:19,13,18/08/2021 15:53:19,VENCIDO,VENCIDO
5,17/08/2021 12:53:19,13,18/08/2021 15:53:19,VENCIDO,VENCIDO
6,17/08/2021 12:53:19,8,18/08/2021 10:53:19,VENCIDO,VENCIDO
7,17/08/2021 12:53:19,10,18/08/2021 12:53:19,VENCIDO,VENCIDO
8,17/08/2021 12:53:19,33,20/08/2021 15:53:19,VENCIDO,VENCIDO
9,17/08/2021 12:53:19,30,20/08/2021 12:53:19,VENCIDO,VENCIDO
